In [1]:
from PIL import Image
from IPython.display import Image as DPImage
import os
import numpy as np
from random import sample
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as sk
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import random
import csv
import skimage

seed = 2023
np.random.seed(seed)  

In [34]:
def display_img(image, title=''):
    plt.title(title)
    plt.imshow(tf.keras.utils.array_to_img(image))
    plt.axis('off')
    plt.show()

In [152]:
isImageGrayscale = lambda img: all(map(lambda p: p[0] == p[1] == p[2], img.getdata()))
                                   
def preprocessAndStoreImages(data_dir, image_dir, save_dir, contrast_ratio = 1.5, threshold = 0.6):
    image_ids = os.listdir(data_dir)
    for _, imgID in enumerate(image_ids):
        full_img_dir = f'{data_dir}/{imgID}/{image_dir}'
        
        # Load Image
        image = Image.open(f"{full_img_dir}/{os.listdir(full_img_dir)[0]}").convert('RGB')
        grayscale = isImageGrayscale(image)
        
        # Convert to grayscale
        image = np.array(image.convert('L'))
        
        # Invert Image if original was colored or background treshold is reached
        if not grayscale or np.sum(image/255) > threshold*image.shape[0]*image.shape[1]:
            image = 255 - image
        
        # Normalize
        image = image - np.min(image)
        if np.max(image) > 0:
            image = np.round(image * (255 / np.max(image))).astype(np.uint8)
            
            # Increase Contrast
            image = np.array(tf.image.adjust_contrast(image.reshape((*image.shape, 1)), contrast_ratio)).reshape(image.shape)
        
        # Save Preprocessed Image
        save_path = f"{data_dir}/{imgID}/{save_dir}"
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        Image.fromarray(image, 'L').save(f"{save_path}/{imgID}.png")

In [155]:
preprocessAndStoreImages("./data/stage1_train", "images", "preprocessed_images")
preprocessAndStoreImages("./data/stage1_test", "images", "preprocessed_images")
preprocessAndStoreImages("./data/stage2_test_final", "images", "preprocessed_images")

In [223]:
def cropImages(data_dir, image_dir, save_dir, window_shape = (128, 128)):
    image_ids = os.listdir(data_dir)
    for tmp, imgID in enumerate(image_ids):
        # Load Image
        full_img_dir = f'{data_dir}/{imgID}/{image_dir}'
        image = Image.open(f"{full_img_dir}/{os.listdir(full_img_dir)[0]}")
        
        # Create Save Dir
        save_path = f"{data_dir}/{imgID}/{save_dir}"
        if not os.path.exists(save_path):
            os.mkdir(save_path)
       
        # Crop Image
        i = 0
        offset_y = 0
        while offset_y < image.size[1]:
            offset_x = 0
            while offset_x < image.size[0]:
                c = np.array(image.crop((
                    offset_x, offset_y,
                    window_shape[0] + offset_x if offset_x+window_shape[0] <= image.size[0] else image.size[0],
                    window_shape[1] + offset_y if offset_y+window_shape[1] <= image.size[1] else image.size[1]
                )))
                X_tmp = np.zeros((window_shape[1], window_shape[0]), dtype=c.dtype)
                X_tmp[:c.shape[0],:c.shape[1]] = c
                
                # Store cropping
                Image.fromarray(X_tmp, 'L').save(f"{save_path}/{i:03d}.png")

                i += 1
                offset_x += window_shape[0]
            offset_y += window_shape[1]

In [ ]:
cropImages("./data/stage1_train", "preprocessed_images", "image_croppings")
cropImages("./data/stage1_train", "combined_masks", "mask_croppings")

cropImages("./data/stage1_test", "preprocessed_images", "image_croppings")
cropImages("./data/stage1_test", "combined_masks", "mask_croppings")

cropImages("./data/stage2_test_final", "preprocessed_images", "image_croppings")